In [10]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

def extract_email(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, timeout=10, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        mailto_links = [a['href'][7:] for a in soup.find_all('a', href=True) if a['href'].startswith('mailto:')]
        if mailto_links:
            return mailto_links[0]

        emails = re.findall(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", soup.text)
        return emails[0] if emails else "No email found"
    except Exception as e:
        return "Failed to access website"

def generate_email(company_name, recipient_email):
    return f"""Subject: Potential Strategic Collaboration with Caprae Capital

Dear {company_name} Team,

I hope this message finds you well.

My name is [Your Name], representing Caprae Capital. We specialize in strategic growth investments and technological partnerships. We came across your company while researching innovative IT firms in Los Angeles and were impressed by your work.

We’re currently seeking to build synergistic collaborations with forward-thinking companies, and we believe your expertise aligns well with our goals.

Additionally, please note that the contents of this email have been securely saved in a structured Excel file (CSV format) for future reference. The email and all related details are also dynamically displayed in a user-friendly graphical interface for real-time visibility and review.

Could we schedule a short call to explore how we might work together?

Warm regards,
[Your Name]
Caprae Capital
📧 {recipient_email if recipient_email else '[Your Email]'}
"""

def scrape_data(search_term):
    chrome_driver_path = "C:\\Users\\Sahil\\Desktop\\inter\\chromedriver.exe"
    options = Options()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    service = Service(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 15)

    driver.get("https://www.google.com/maps")
    search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
    search_box.send_keys(search_term)
    search_box.send_keys(Keys.RETURN)
    time.sleep(5)

    for _ in range(3):
        driver.execute_script("document.querySelector('div[role=\"main\"]').scrollBy(0, 1000);")
        time.sleep(2)

    results = []
    businesses = driver.find_elements(By.CLASS_NAME, "Nv2PK")

    for i in range(min(5, len(businesses))):
        try:
            ActionChains(driver).move_to_element(businesses[i]).click().perform()
            time.sleep(3)

            try:
                name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "DUwDvf"))).text
            except:
                name = "No name found"

            try:
                website = driver.find_element(By.CSS_SELECTOR, "a[data-item-id='authority']").get_attribute("href")
            except:
                website = "No website listed"

            email = extract_email(website) if website != "No website listed" else "No email"
            cold_email = generate_email(name, email) if email not in ["No email found", "No website listed", "Failed to access website", "No email"] else "No email generated"

            results.append({
                'Company Name': name,
                'Website': website,
                'Email': email,
                'Cold Email': cold_email
            })

            driver.execute_script("window.history.go(-1)")
            time.sleep(3)
            businesses = driver.find_elements(By.CLASS_NAME, "Nv2PK")

        except Exception as e:
            print(f"Error scraping business #{i+1}: {e}")

    driver.quit()
    return results

class WebScraperApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Business Email Scraper")
        self.root.geometry("950x600")
        self.root.config(bg="#f0f4f7")

        tk.Label(root, text="Google Maps Business Scraper", font=("Helvetica", 20, "bold"), fg="#1e90ff", bg="#f0f4f7").pack(pady=20)
        tk.Label(root, text="Enter Search Term (e.g., 'IT companies in LA')", font=("Helvetica", 12), bg="#f0f4f7").pack()

        self.search_entry = tk.Entry(root, width=60, font=("Arial", 12), bd=2, relief="solid", justify="center")
        self.search_entry.pack(pady=10)

        self.scrape_button = tk.Button(root, text="Start Scraping", command=self.scrape, font=("Arial", 14), bg="#4CAF50", fg="white", relief="raised", padx=20, pady=10, bd=0)
        self.scrape_button.pack(pady=20)

        self.scrape_button.bind("<Enter>", lambda e: self.scrape_button.config(bg="#45a049"))
        self.scrape_button.bind("<Leave>", lambda e: self.scrape_button.config(bg="#4CAF50"))

        self.tree = ttk.Treeview(root, columns=("Company Name", "Website", "Email", "Cold Email"), show="headings", height=10)
        self.tree.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

        for col in ("Company Name", "Website", "Email", "Cold Email"):
            self.tree.heading(col, text=col)
            self.tree.column(col, width=200)

        self.tree.bind("<Double-1>", self.show_full_email)

        self.results_data = []

    def scrape(self):
        search_term = self.search_entry.get()
        if not search_term:
            messagebox.showerror("Error", "Please enter a search term.")
            return

        self.results_data = scrape_data(search_term)

        for row in self.tree.get_children():
            self.tree.delete(row)

        for result in self.results_data:
            cold_preview = result['Cold Email'].split('\n')[0] + "..." if result['Cold Email'] else ""
            self.tree.insert("", "end", values=(result['Company Name'], result['Website'], result['Email'], cold_preview))

        df = pd.DataFrame(self.results_data)
        df.to_csv("scraped_data.csv", index=False)
        messagebox.showinfo("Success", "Scraping complete. Data saved to 'scraped_data.csv'. Click on a row to view full email.")

    def show_full_email(self, event):
        selected = self.tree.selection()
        if selected:
            index = self.tree.index(selected)
            full_email = self.results_data[index]["Cold Email"]
            popup = tk.Toplevel()
            popup.title("Cold Email Content")
            popup.geometry("600x400")
            text = tk.Text(popup, wrap=tk.WORD)
            text.insert(tk.END, full_email)
            text.pack(expand=True, fill=tk.BOTH)
            text.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = WebScraperApp(root)
    root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

def extract_email(url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, timeout=10, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        mailto_links = [a['href'][7:] for a in soup.find_all('a', href=True) if a['href'].startswith('mailto:')]
        if mailto_links:
            return mailto_links[0]

        emails = re.findall(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", soup.text)
        return emails[0] if emails else "No email found"
    except Exception as e:
        return "Failed to access website"

def generate_email(company_name, recipient_email):
    return f"""Subject: Potential Strategic Collaboration with Caprae Capital

Dear {company_name} Team,

I hope this message finds you well.

My name is [Your Name], representing Caprae Capital. We specialize in strategic growth investments and technological partnerships. We came across your company while researching innovative IT firms in Los Angeles and were impressed by your work.

We’re currently seeking to build synergistic collaborations with forward-thinking companies, and we believe your expertise aligns well with our goals.

Additionally, please note that the contents of this email have been securely saved in a structured Excel file (CSV format) for future reference. The email and all related details are also dynamically displayed in a user-friendly graphical interface for real-time visibility and review.

Could we schedule a short call to explore how we might work together?

Warm regards,
[Your Name]
Caprae Capital
📧 {recipient_email if recipient_email else '[Your Email]'}
"""

def scrape_data(search_term):
    chrome_driver_path = "C:\\Users\\Sahil\\Desktop\\inter\\chromedriver.exe"
    options = Options()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    service = Service(executable_path=chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=options)
    wait = WebDriverWait(driver, 15)

    driver.get("https://www.google.com/maps")
    search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
    search_box.send_keys(search_term)
    search_box.send_keys(Keys.RETURN)
    time.sleep(5)

    for _ in range(3):
        driver.execute_script("document.querySelector('div[role=\"main\"]').scrollBy(0, 1000);")
        time.sleep(2)

    results = []
    businesses = driver.find_elements(By.CLASS_NAME, "Nv2PK")

    for i in range(min(5, len(businesses))):
        try:
            ActionChains(driver).move_to_element(businesses[i]).click().perform()
            time.sleep(3)

            try:
                name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "DUwDvf"))).text
            except:
                name = "No name found"

            try:
                website = driver.find_element(By.CSS_SELECTOR, "a[data-item-id='authority']").get_attribute("href")
            except:
                website = "No website listed"

            email = extract_email(website) if website != "No website listed" else "No email"
            cold_email = generate_email(name, email) if email not in ["No email found", "No website listed", "Failed to access website", "No email"] else "No email generated"

            results.append({
                'Company Name': name,
                'Website': website,
                'Email': email,
                'Cold Email': cold_email
            })

            driver.execute_script("window.history.go(-1)")
            time.sleep(3)
            businesses = driver.find_elements(By.CLASS_NAME, "Nv2PK")

        except Exception as e:
            print(f"Error scraping business #{i+1}: {e}")

    driver.quit()
    return results

class WebScraperApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Business Email Scraper")
        self.root.geometry("950x600")
        self.root.config(bg="#f0f4f7")

        tk.Label(root, text="Google Maps Business Scraper", font=("Helvetica", 20, "bold"), fg="#1e90ff", bg="#f0f4f7").pack(pady=20)
        tk.Label(root, text="Enter Search Term (e.g., 'IT companies in LA')", font=("Helvetica", 12), bg="#f0f4f7").pack()

        self.search_entry = tk.Entry(root, width=60, font=("Arial", 12), bd=2, relief="solid", justify="center")
        self.search_entry.pack(pady=10)

        self.scrape_button = tk.Button(root, text="Start Scraping", command=self.scrape, font=("Arial", 14), bg="#4CAF50", fg="white", relief="raised", padx=20, pady=10, bd=0)
        self.scrape_button.pack(pady=20)

        self.scrape_button.bind("<Enter>", lambda e: self.scrape_button.config(bg="#45a049"))
        self.scrape_button.bind("<Leave>", lambda e: self.scrape_button.config(bg="#4CAF50"))

        self.tree = ttk.Treeview(root, columns=("Company Name", "Website", "Email", "Cold Email"), show="headings", height=10)
        self.tree.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)

        for col in ("Company Name", "Website", "Email", "Cold Email"):
            self.tree.heading(col, text=col)
            self.tree.column(col, width=200)

        self.tree.bind("<Double-1>", self.show_full_email)

        self.results_data = []

    def scrape(self):
        search_term = self.search_entry.get()
        if not search_term:
            messagebox.showerror("Error", "Please enter a search term.")
            return

        self.results_data = scrape_data(search_term)

        for row in self.tree.get_children():
            self.tree.delete(row)

        for result in self.results_data:
            cold_preview = result['Cold Email'].split('\n')[0] + "..." if result['Cold Email'] else ""
            self.tree.insert("", "end", values=(result['Company Name'], result['Website'], result['Email'], cold_preview))

        df = pd.DataFrame(self.results_data)
        df.to_csv("scraped_data.csv", index=False)
        messagebox.showinfo("Success", "Scraping complete. Data saved to 'scraped_data.csv'. Click on a row to view full email.")

    def show_full_email(self, event):
        selected = self.tree.selection()
        if selected:
            index = self.tree.index(selected)
            full_email = self.results_data[index]["Cold Email"]
            popup = tk.Toplevel()
            popup.title("Cold Email Content")
            popup.geometry("600x400")
            text = tk.Text(popup, wrap=tk.WORD)
            text.insert(tk.END, full_email)
            text.pack(expand=True, fill=tk.BOTH)
            text.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = WebScraperApp(root)
    root.mainloop()
